In [1]:
# Imports
import random
import numpy as np
import time as t

import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import gradcheck
import torchvision.utils as vutils
import time as time

from torch import autograd

import model_netD_regression as model
import keijzer_exogan as ke

# initialize random seeds
manualSeed = 999
random.seed(manualSeed)
torch.manual_seed(manualSeed) 
#torch.set_num_threads(16)

"""
Local variables
"""
selected_gpus = [0,1] # Selected GPUs

path = '/datb/16011015/ExoGAN_data/selection//' # Storage location of the train/test data

print('Loading data...')
images = np.load(path+'first_chunks_25_percent_images_v4.1.npy').astype('float32')

images = images[:512] # select first ... images

use_saved_weights = False

"""
Local variables that generally stay unchanged
"""
batch_size = 64 # 64
num_epochs = 100

lr = 1e-4

beta1 = 0.5 # beta1 for Adam
beta2 = 0.9 # beta2 for Adam

workers = 0 # Number of workers for dataloader, 0 when to_vram is enabled
image_size = 32

torch.backends.cudnn.benchmark=True # Uses udnn auto-tuner to find the best algorithm to use for your hardware, speeds up training by almost 50%


print('Batch size: ', batch_size)
ngpu = len(selected_gpus)
print('Number of GPUs used: ', ngpu)

"""
Load data and prepare DataLoader
"""
shuffle = True

if shuffle:
    np.random.shuffle(images) # shuffles the images

print('Number of images: ', len(images))

dataset = ke.numpy_dataset(data=images, to_vram=True) # to_vram pins it to all GPU's

# Create the dataloader
dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size,
                                         shuffle=True, num_workers=workers, pin_memory=False)


"""
Load and setup models
"""
print('Initializing cuda...')
# Initialize cuda
device = torch.device("cuda:"+str(selected_gpus[0]) if (torch.cuda.is_available() and ngpu > 0) else "cpu")

# Load models
netD = model.Discriminator(ngpu).to(device)

# Apply weights

# custom weights initialization called on netG and netD
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        nn.init.normal_(m.weight.data, 1.0, 0.02)
        nn.init.constant_(m.bias.data, 0)

netD.apply(weights_init)

if use_saved_weights:
    try:
        # Load saved weights
        netD.load_state_dict(torch.load('gan_data//weights//netD_regression', map_location=device))
        print('Succesfully loaded saved weights.')
    except:
        print('Could not load saved weights, using new ones.')
        pass

# Handle multi-gpu if desired
if (device.type == 'cuda') and (ngpu > 1):
    netD = nn.DataParallel(netD, device_ids=selected_gpus, output_device=device)
    
    
"""
Define input training stuff (fancy this up)
"""
# Setup Adam optimizers for both G and D
optimizerD = optim.Adam(netD.parameters(), lr=lr, betas=(beta1, beta2)) # should be sgd

def save_progress(experiment_name, variable_name, progress_list):
    path = 'gan_data//training_progress//'
    progress_list = np.array(progress_list).astype('float32')
    #file = open(path+variable_name+"_"+experiment_name+'.npy', mode="a")
    file_name = path+variable_name+"_"+experiment_name+'.npy'
    file = np.load(file_name, mmap_mode='r+')
    file = np.append(file, progress_list)
    np.save(file_name, file)
    #file.close()

Loading data...
Batch size:  64
Number of GPUs used:  2
Number of images:  512
Initializing cuda...


In [ ]:
MSELoss = nn.MSELoss()
val_loss = 99999
t1 = time.time()
for epoch in range(num_epochs):
    for i, data in enumerate(dataloader, 0):
        #print('='*15)
        aspa = data.to(device)
        b_size = aspa.size(0)

        """Decode real values from aspa"""
        #magnitude = [1e27, 1e3, 1, 1e7, 1, 1, 1]
        params_real = []
        for j in range(b_size):
            params = [aspa[j][:16, 25+i:26+i].mean() for i in range(7)] # pixel values of normalized params in the aspa
            params_real.append(params)

        real = torch.tensor(params_real).to(device) # shape (b_size, params)

        #print('real: %s' % real)
        
        
        if i < 0.7*len(dataloader):
            """
            Train D on train data
            """
            for p in netD.parameters():
                p.requires_grad_(True)

            netD.zero_grad()
            output = netD(aspa) # shape (b_size, params)
            #print('predicted: %s' % output)
            train_loss = MSELoss(output, real)
            train_loss.backward()
            optimizerD.step()
        else:
            """
            Validate on val data
            """
            output = netD(aspa)
            val_loss = MSELoss(output, real)
    
    
        """
        Print statistics
        """
        train_loss.detach().cpu()
        #print(loss)

        if i % (8) == 0:
            t2 = time.time()
            print('[%d/%d][%d/%d] \t train_loss = %.5f \t val_loss = %.5f \t t = %.3f' % 
                      (epoch, num_epochs, i, len(dataloader), train_loss, val_loss, (t2-t1) ))
            t1 = time.time()

[0/100][0/8] 	 train_loss = 0.00093 	 val_loss = 99999.00000 	 t = 0.190
[1/100][0/8] 	 train_loss = 0.00223 	 val_loss = 0.00220 	 t = 1.200
[2/100][0/8] 	 train_loss = 0.00296 	 val_loss = 0.00281 	 t = 1.202
[3/100][0/8] 	 train_loss = 0.00110 	 val_loss = 0.00109 	 t = 1.206
[4/100][0/8] 	 train_loss = 0.00013 	 val_loss = 0.00018 	 t = 1.202
[5/100][0/8] 	 train_loss = 0.00480 	 val_loss = 0.00476 	 t = 1.203
[6/100][0/8] 	 train_loss = 0.00179 	 val_loss = 0.00146 	 t = 1.194


In [25]:
path = '/datb/16011015/ExoGAN_data/selection//' #notice how you dont put the last folder in here...
images = np.load(path+'last_chunks_25_percent_images_v4.1.npy') # 4.1 is a random selection of 5k images
print('Loaded %s images' % len(images))

Loaded 5000 images


In [26]:
aspa = torch.tensor(images[0]).reshape(1,1,32,32).to(device).float()

output = netD(aspa).view(-1).detach().cpu().numpy()
output

array([0.5424745 , 0.4580737 , 0.45168817, 0.44521517, 0.5125383 ,
       0.42110914, 0.4804378 ], dtype=float32)

In [27]:
aspa = aspa.detach().cpu().numpy().reshape(32,32)
reals = np.array([aspa[:16, 25+i:26+i].mean() for i in range(7)])
reals

array([-0.11111111, -1.        ,  0.33333334,  0.7777778 ,  0.5555556 ,
       -0.11111111,  0.11111111], dtype=float32)

In [28]:
predictions = ke.decode_params_from_list(output)
predictions

{'ch4_mixratio': -2.9190914034843445,
 'co2_mixratio': -3.0261180102825165,
 'co_mixratio': -2.818467751145363,
 'h2o_mixratio': -2.7061159014701843,
 'planet_mass': 3.274969970035553e+27,
 'planet_radius': 91291553.1051606,
 'temp_profile': 1729.0368527173996}

In [29]:
reals = ke.decode_params_from_list(params)
reals

{'ch4_mixratio': -3.333333298563957,
 'co2_mixratio': -4.888888891786337,
 'co_mixratio': -4.111111108213663,
 'h2o_mixratio': -2.5555554628372192,
 'planet_mass': 2.5306666657239197e+27,
 'planet_radius': 99428978.10187936,
 'temp_profile': 1000.0}